# Imports

In [ ]:
import numpy as np
import pandas as pd
import zarr
import dask
from dask import delayed
from dask.distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
import streamz
import holoviews as hv
from holoviews.streams import Stream, param
from bokeh.models.tools import HoverTool
import matplotlib.pyplot as plt
from tqdm import tnrange, tqdm, tqdm_notebook
import warnings
from functools import partial
from cytoolz import compose
from operator import getitem
import nd2reader
from importlib import reload

In [ ]:
#%load_ext autoreload
#%autoreload 2

In [ ]:
# from processing import *
# from trench_detection import *
# from trench_segmentation import *
# from trench_segmentation.watershed import *
# from util import *
# from ui import *
import common, trench_detection, util, ui

In [ ]:
%load_ext line_profiler
hv.extension("bokeh")
%matplotlib inline
tqdm.monitor_interval = 0

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="1:00:00",
    # job_extra=['-p transfer'],
    # job_extra=['--cores-per-socket=8'],
    # interface='ib0',
    memory="64GB",
    # local_directory='/tmp',
    threads=1,
    processes=1,
    # diagnostics_port=('127.0.0.1', 8787),
    env_extra=['export PYTHONPATH="/home/jqs1/projects/matriarch"'],
)
client = Client(cluster)

In [ ]:
cluster._widget().children[1].children[1].children[0].children[0].layout.width = "200px"
cluster

In [ ]:
cluster.stop_workers(cluster.jobs)

In [ ]:
cluster = LocalCluster(
    n_workers=0, ncores=1, processes=True, ip="0.0.0.0"
)  # TODO: ncores ignored
cluster.start_worker(ncores=1)
client = Client(cluster)

In [ ]:
LocalCluster?

In [ ]:
client

# Functions

# Fix diag

In [ ]:
def serialize_holoviews(x):
    print("z")
    header = {}
    frames = [hv.Store.dumps(x)]
    return header, frames


def deserialize_holoviews(header, frames):
    print("h")
    if len(frames) > 1:  # this may be cut up for network reasons
        frame = "".join(frames)
    else:
        frame = frames[0]
    return hv.Store.loads(frame)


from distributed.protocol.serialize import register_serialization

register_serialization(
    hv.element.chart.Scatter, serialize_holoviews, deserialize_holoviews
)

In [ ]:
reload(hv.core.dimension)

In [ ]:
hv.core.dimension.LabelledData

In [ ]:
orig_getstate = hv.core.dimension.LabelledData.__getstate__
orig_setstate = hv.core.dimension.LabelledData.__setstate__

In [ ]:
from functools import wraps

# orig_getstate = hv.Dimension.__getstate__
# orig_setstate = hv.Dimension.__setstate__


@wraps(orig_getstate)
def new_getstate(self):
    hv.Store.save_option_state = True
    val = orig_getstate(self)
    # hv.Store.save_option_state = False
    return val


# @wraps(orig_setstate)
def new_setstate(self, d):
    print("NEW SETSTATE")
    hv.Store.load_counter_offset = hv.StoreOptions.id_offset()
    orig_setstate(self, d)
    # hv.Store.load_counter_offset = None


# hv.core.dimension.LabelledData.__getstate__ = new_getstate
# hv.core.dimension.LabelledData.__getstate__ = orig_getstate
hv.core.dimension.LabelledData.__setstate__ = new_setstate
# hv.core.dimension.LabelledData.__setstate__ = orig_setstate

In [ ]:
hv.Store.save_option_state = True

In [ ]:
a = hv.Scatter(np.random.random(10)).options(
    color="yellow", size=10, backend="bokeh", clone=False
)

In [ ]:
a

In [ ]:
import pickle

In [ ]:
hv.__file__

In [ ]:
b = pickle.loads(pickle.dumps(a))
b

In [ ]:
b.id = 3

In [ ]:
b

In [ ]:
b.__dict__

In [ ]:
z = pickle.dumps(a)
hv.Store._custom_options["bokeh"] = {}
pickle.loads(z)

In [ ]:
z = hv.Store.dumps(a)
hv.Store._custom_options["bokeh"] = {}
hv.Store.loads(z)

In [ ]:
hv.Store._custom_options

In [ ]:
import pickle

In [ ]:
z = pickle.dumps(a)

In [ ]:
hv.Store._custom_options["bokeh"] = {}

In [ ]:
hv.Store.load_counter_offset = hv.StoreOptions.id_offset()
pickle.loads(z)

In [ ]:
import common

In [ ]:
reload(common)

In [ ]:
hv.Dimension.__setstate__

In [ ]:
def test():
    import common

    # hv.extension('bokeh')
    # hv.notebook_extension('bokeh')
    # hv.Store.save_option_state = True
    val = hv.Scatter(np.random.random(10)).options(color="green", size=5)
    # import pickle
    # return pickle.dumps(val)
    return val


list(client.run(test).values())[0]  # ['tcp://10.120.17.16:34432']

In [ ]:
import pickle

In [ ]:
pickle.loads(list(client.run(test).values())[0])

In [ ]:
list(client.run(test).values())[0]

In [ ]:
hv.Store.load_counter_offset = hv.StoreOptions.id_offset()

In [ ]:
hv.Store._custom_options

In [ ]:
from dask.distributed import protocol  # as s#.typename(type(a))

In [ ]:
protocol.serialize

In [ ]:
type(a).__module__

In [ ]:
a.print_param_defaults()

In [ ]:
a.get_param_values()

In [ ]:
hv.Store.transfer_options?

In [ ]:
a.

In [ ]:
a

In [ ]:
a.options?

In [ ]:
b = a.options(color="red", size=10)
b

In [ ]:
import pickle

In [ ]:
hv.Store.loads(hv.Store.dumps(b))

In [ ]:
client.close()

In [ ]:
client = Client(cluster)

In [ ]:
reload(trench_detection)

In [ ]:
diag = util.tree()
_ = trench_detection.get_trenches(
    partial(get_frame, v=10, c=2)(nd2_filenames[0]), diagnostics=diag
)

In [ ]:
diag["label_1"]["trench_anchors"]["trench_anchor_profile"]

# Loading data

In [ ]:
nd2_filenames = [
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2",
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2",
]

In [ ]:
nd2_sizes = {
    filename: nd2reader.ND2Reader(filename).sizes for filename in nd2_filenames
}

In [ ]:
nd2s = {
    filename: client.submit(nd2reader.ND2Reader, filename, memmap=False)
    for filename in nd2_filenames
}
# nd2s = {filename: nd2reader.ND2Reader(filename, memmap=False) for filename in nd2_filenames}

In [ ]:
# nd2_sizes = util.gather_futures(client, util.map_futures(partial(client.submit, lambda nd2: nd2.sizes), nd2s))

In [ ]:
def do_reload():
    from importlib import reload

    reload(util)
    reload(trench_detection)


client.run(do_reload)

In [ ]:
reload(trench_detection)

In [ ]:
def get_frame(nd2_filename=None, v=None, c=None):
    return nd2reader.ND2Reader(nd2_filename).get_frame_2D(v=v, c=c, memmap=False)

In [ ]:
a = client.submit(
    compose(
        util.wrap_diagnostics(trench_detection.get_trenches),
        lambda x: x.get_frame_2D(v=0, c=2, memmap=False),
    ),
    nd2s[nd2_filenames[0]],
)

In [ ]:
z = a.result()[1]

In [ ]:
ui.display_plot_browser(z)

In [ ]:
ui.display_plot_browser(a.result()[1])

In [ ]:
client.submit(
    compose(
        util.wrap_diagnostics(trench_detection.get_trenches),
        lambda x: ND2Reader(x).get_frame_2D(v=0, c=2, memmap=False),
    ),
    nd2_filenames[0],
)

In [ ]:
_.result()

In [ ]:
trench_data = {
    filename: {
        v: client.submit(
            util.wrap_diagnostics(trench_detection.get_trenches),
            client.submit(partial(get_frame, v=v, c=sizes["c"] - 1), filename),
        )
        for v in range(2)
    }  # sizes['v']//50)}
    for filename, nd2, sizes in util.zip_dicts(nd2s, nd2_sizes)
}

In [ ]:
trench_data = {
    filename: {
        v: client.submit(
            compose(
                util.wrap_diagnostics(trench_detection.get_trenches),
                lambda x: x.get_frame_2D(v=v, c=sizes["c"] - 1, memmap=False),
            ),
            nd2,
        )
        for v in range(sizes["v"] // 10)
    }
    for filename, nd2, sizes in util.zip_dicts(nd2s, nd2_sizes)
}

In [ ]:
trench_data["/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2"][4]

In [ ]:
client.cancel(trench_data)

In [ ]:
progress(trench_data)

In [ ]:
res = trench_data["/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2"][0].result()[1]

In [ ]:
a = res["label_1"]["trench_anchors"]["trench_anchor_profile"]

In [ ]:
a

In [ ]:
a.Scatter.I.

In [ ]:
a.Scatter.I.options(color="red", size=5)

In [ ]:
ui.display_plot_browser(
    trench_data["/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2"][14].result()[1]
)

In [ ]:
from traceback import print_tb, print_exception
import traceback

In [ ]:
traceback.extract_tb(t)

# Scratch

In [ ]:
def load_nd2(filename):
    nd2 = nd2reader.ND2Reader(filename)
    diag = tree()
    return get_trenches(nd2[1, 30], diagnostics=diag_pos[pos])

In [ ]:
trench_positions = {}

# Old

In [ ]:
get_trenches(root_group["raw"][str(pos)][1, 30], diagnostics=diag_pos[pos])

In [ ]:
diag = tree()
_ = get_trenches(root_group["raw"][str(pos)][0, 1], diagnostics=diag)

In [ ]:
def f(img_stack):
    ary = np.stack(
        [
            segment_trench(img_stack[t], diagnostics=None)
            for t in range(img_stack.shape[0])
        ],
        axis=0,
    )
    ary = zarr.array(ary, compressor=DEFAULT_FRAME_COMPRESSOR)
    return ary


trench_seg_masks = positionwise_trenchwise_map(
    root_group["raw"],
    trench_points_pos,
    f,
    channel_slice=1,
    preload=True,
    time_slice=slice(None),
    positions=range(1),
)

In [ ]:
def f(img_stack):
    return pd.Series(np.percentile(img_stack, 95, axis=(1, 2)))
    # return pd.Series(np.max(img_stack, axis=(1,2)))


trench_traces_all = positionwise_trenchwise_map(
    root_group["raw"],
    trench_points_pos,
    f,
    channel_slice=2,
    preload=True,
    time_slice=slice(None),
    positions=range(100),
)